In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import networkx as nx
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import random
import pickle

print("Libraries imported successfully.")


Libraries imported successfully.


In [8]:
# Section 2: Preprocessing and Graph Construction

# File paths
original_features_path = "../foursquare_data/features.json"
original_edges_path = "../foursquare_data/edges.csv"

# Load data
def load_data():
    print("Loading original features and edges...")
    with open(original_features_path, "r") as f:
        features = json.load(f)
    print(f"Loaded {len(features)} nodes with features.")
    
    edges = pd.read_csv(
        original_edges_path, names=["source", "target"], skiprows=1
    )  # Skip header row
    print(f"Loaded {len(edges)} edges.")
    return features, edges

# Create graph and extract largest connected component
def create_graph(features, edges):
    print("Creating graph from edges...")
    edges["source"] = edges["source"].astype(str)
    edges["target"] = edges["target"].astype(str)
    G = nx.from_pandas_edgelist(edges, source="source", target="target")
    print(f"Graph created with {len(G.nodes)} nodes and {len(G.edges)} edges.")

    print("Identifying the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc).copy()
    print(f"Largest connected component has {len(G_lcc.nodes)} nodes and {len(G_lcc.edges)} edges.")

    print("Assigning features to nodes...")
    for node, feats in features.items():
        if node in G_lcc.nodes:
            G_lcc.nodes[node]["features"] = feats
    print("Node features assigned.")
    
    return G_lcc

def standardize_features(G, output_dim=128):
    print("Standardizing features to fixed dimensions...")

    # Extract nodes with features
    nodes_with_features = [
        node for node, feats in nx.get_node_attributes(G, "features").items() if feats
    ]
    print(f"Number of nodes with features: {len(nodes_with_features)}")

    # Extract feature lists for nodes with features
    feature_list = [
        set(G.nodes[node]["features"]) for node in nodes_with_features
    ]
    
    mlb = MultiLabelBinarizer()
    binary_features = mlb.fit_transform(feature_list)
    print(f"Initial feature matrix shape: {binary_features.shape}")

    # Apply PCA if necessary
    if binary_features.shape[1] > output_dim:
        print(f"Reducing dimensions to {output_dim} using PCA...")
        pca = PCA(n_components=output_dim)
        reduced_features = pca.fit_transform(binary_features)
        print(f"Feature matrix shape after PCA: {reduced_features.shape}")
    else:
        print(f"No dimensionality reduction needed. Retaining shape {binary_features.shape}")
        reduced_features = binary_features

    # Assign standardized features back to the corresponding nodes
    print("Assigning standardized features back to nodes...")
    for idx, node in enumerate(nodes_with_features):
        G.nodes[node]["features"] = reduced_features[idx]
    
    print("Feature standardization complete.")



# Load, process, and standardize graph
print("Starting graph preprocessing...")
features, edges = load_data()
G = create_graph(features, edges)
standardize_features(G, output_dim=128)
print(f"Graph preprocessing complete. Final graph has {len(G.nodes)} nodes and {len(G.edges)} edges.")


Starting graph preprocessing...
Loading original features and edges...
Loaded 30075 nodes with features.
Loaded 701311 edges.
Creating graph from edges...
Graph created with 114324 nodes and 701311 edges.
Identifying the largest connected component...
Largest connected component has 111251 nodes and 699461 edges.
Assigning features to nodes...
Node features assigned.
Standardizing features to fixed dimensions...
Number of nodes with features: 2847
Initial feature matrix shape: (2847, 10141)
Reducing dimensions to 128 using PCA...
Feature matrix shape after PCA: (2847, 128)
Assigning standardized features back to nodes...
Feature standardization complete.
Graph preprocessing complete. Final graph has 111251 nodes and 699461 edges.


In [9]:
def check_connectivity_bfs(G):
    print("Performing BFS to ensure all nodes are connected...")
    start_node = next(iter(G.nodes))  # Get an arbitrary starting node
    visited = set()
    queue = [start_node]

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            queue.extend(neighbor for neighbor in G.neighbors(node) if neighbor not in visited)

    if len(visited) == len(G.nodes):
        print("All nodes are connected. The graph is a single connected component.")
    else:
        print(f"Graph is not fully connected. Only {len(visited)} out of {len(G.nodes)} nodes are reachable.")


check_connectivity_bfs(G)  # Ensure the graph is a single connected component

Performing BFS to ensure all nodes are connected...
All nodes are connected. The graph is a single connected component.


In [12]:
# Section 3A: Create Feature Vectors

def create_feature_vectors(G, edges):
    print("Creating feature vectors for ML tasks...")
    X, y = [], []

    print("Processing positive samples (existing edges)...")
    for i, (_, row) in enumerate(edges.iterrows()):
        node1, node2 = str(row["source"]), str(row["target"])
        if (
            node1 in G.nodes 
            and node2 in G.nodes 
            and "features" in G.nodes[node1] 
            and "features" in G.nodes[node2]
        ):
            feature_vector = np.array(G.nodes[node1]["features"]) - np.array(G.nodes[node2]["features"])
            X.append(feature_vector)
            y.append(1)
        if i % 50000 == 0:
            print(f"Processed {i} positive samples.")

    print("Generating negative samples (random non-existing edges)...")
    all_nodes = [node for node in G.nodes if "features" in G.nodes[node]]  # Nodes with features
    for i in range(len(edges)):
        node1, node2 = np.random.choice(all_nodes, 2, replace=False)
        if not G.has_edge(node1, node2):
            feature_vector = np.array(G.nodes[node1]["features"]) - np.array(G.nodes[node2]["features"])
            X.append(feature_vector)
            y.append(0)
        if i % 50000 == 0:
            print(f"Generated {i} negative samples.")

    print(f"Feature vectors created. Total samples: {len(X)}")
    return np.array(X), np.array(y)


# Create and split feature vectors
print("Creating and splitting feature vectors...")
X, y = create_feature_vectors(G, edges)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

with open("feature_vectors.pkl", "wb") as f:
    pickle.dump((X, y, X_train, X_test, y_train, y_test), f)

print("Feature vectors and splits saved successfully.")


Creating and splitting feature vectors...
Creating feature vectors for ML tasks...
Processing positive samples (existing edges)...
Processed 0 positive samples.
Processed 50000 positive samples.
Processed 100000 positive samples.
Processed 150000 positive samples.
Processed 200000 positive samples.
Processed 250000 positive samples.
Processed 300000 positive samples.
Processed 350000 positive samples.
Processed 400000 positive samples.
Processed 450000 positive samples.
Processed 500000 positive samples.
Processed 550000 positive samples.
Processed 600000 positive samples.
Processed 650000 positive samples.
Processed 700000 positive samples.
Generating negative samples (random non-existing edges)...
Generated 0 negative samples.
Generated 50000 negative samples.
Generated 100000 negative samples.
Generated 150000 negative samples.
Generated 200000 negative samples.
Generated 250000 negative samples.
Generated 300000 negative samples.
Generated 350000 negative samples.
Generated 400000 

In [13]:
with open("feature_vectors.pkl", "rb") as f:
    X, y, X_train, X_test, y_train, y_test = pickle.load(f)

print("Feature vectors and splits loaded successfully.")
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

Feature vectors and splits loaded successfully.
Training set size: 565492, Test set size: 141373


In [14]:
# Section 3B: Train the Neural Network

# Define the neural network
print("Defining the neural network model...")
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],), name="Input_Layer"),
    tf.keras.layers.Dense(64, activation="relu", name="Hidden_Layer_1"),
    tf.keras.layers.Dense(32, activation="relu", name="Hidden_Layer_2"),
    tf.keras.layers.Dense(1, activation="sigmoid", name="Output_Layer"),
])
print("Model defined successfully.")
model.summary()

# Compile the model
print("Compiling the model...")
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print("Model compiled successfully.")

# Define a custom callback for logging
class TrainingLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\nEpoch {epoch + 1}:")
        print(
            f"  Training Loss: {logs['loss']:.4f}, Training Accuracy: {logs['accuracy']:.4f}"
        )
        print(
            f"  Validation Loss: {logs['val_loss']:.4f}, Validation Accuracy: {logs['val_accuracy']:.4f}"
        )

# Train the model
print("Starting model training...")
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[TrainingLogger()],
    verbose=0  # Suppress default verbose to use custom logging
)
print("Model training complete.")

# Evaluate the model
model.save("trained_model.h5")
print("Model saved successfully to 'trained_model.h5'.")

# Save the training history
with open("training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)
print("Training history saved successfully to 'training_history.pkl'.")

# Evaluate the model
print("Evaluating the model on the test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Defining the neural network model...


/Users/milesbramwit/cs6850/Final_Project/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model defined successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (Dense)             │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_1 (Dense)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,881 (105.00 KB)

 Trainable params: 26,881 (105.00 KB)

 Non-trainable params: 0 (0.00 B)

Compiling the model...
Model compiled successfully.
Starting model training...

Epoch 1:
  Training Loss: 0.0550, Training Accuracy: 0.9904
  Validation Loss: 0.0538, Validation Accuracy: 0.9902

Epoch 2:
  Training Loss: 0.0515, Training Accuracy: 0.9904
  Validation Loss: 0.0530, Validation Accuracy: 0.9902

Epoch 3:
  Training Loss: 0.0498, Training Accuracy: 0.9904
  Validation Loss: 0.0525, Validation Accuracy: 0.9902

Epoch 4:
  Training Loss: 0.0483, Training Accuracy: 0.9904
  Validation Loss: 0.0525, Validation Accuracy: 0.9902

Epoch 5:
  Training Loss: 0.0470, Training Accuracy: 0.9904
  Validation Loss: 0.0529, Validation Accuracy: 0.9902

Epoch 6:
  Training Loss: 0.0458, Training Accuracy: 0.9904
  Validation Loss: 0.0537, Validation Accuracy: 0.9902

Epoch 7:
  Training Loss: 0.0446, Training Accuracy: 0.9905
  Validation Loss: 0.0552, Validation Accuracy: 0.9902

Epoch 8:
  Training Loss: 0.0434, Training Accuracy: 0.9905
  Validation Loss: 0.0561, Validation Accuracy: 


Epoch 20:
  Training Loss: 0.0346, Training Accuracy: 0.9918
  Validation Loss: 0.0914, Validation Accuracy: 0.9891
Model training complete.
Model saved successfully to 'trained_model.h5'.
Training history saved successfully to 'training_history.pkl'.
Evaluating the model on the test set...
4418/4418 ━━━━━━━━━━━━━━━━━━━━ 2s 376us/step - accuracy: 0.9901 - loss: 0.0800
Test Loss: 0.0849
Test Accuracy: 0.9900


In [15]:
# Load the trained model
model = tf.keras.models.load_model("trained_model.h5")
print("Model loaded successfully from 'trained_model.h5'.")
model.summary()

# Load the training history
with open("training_history.pkl", "rb") as f:
    history = pickle.load(f)
print("Training history loaded successfully from 'training_history.pkl'.")

# Print the loaded training history (optional)
print("Loaded Training History:")
for key, values in history.items():
    print(f"{key}: {values[:5]}...")  # Show first 5 values as a preview

Model loaded successfully from 'trained_model.h5'.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (Dense)             │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_1 (Dense)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,883 (105.02 KB)

 Trainable params: 26,881 (105.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Training history loaded successfully from 'training_history.pkl'.
Loaded Training History:
accuracy: [0.9904242753982544, 0.9904308915138245, 0.9904308915138245, 0.9904308915138245, 0.9904220700263977]...
loss: [0.05502021685242653, 0.051535654813051224, 0.04984559491276741, 0.048342760652303696, 0.04700388386845589]...
val_accuracy: [0.9901944398880005, 0.9901944398880005, 0.9901944398880005, 0.9901944398880005, 0.9901944398880005]...
val_loss: [0.0537579283118248, 0.052960436791181564, 0.05247677117586136, 0.052535321563482285, 0.05290927365422249]...


In [ ]:
def predict_next_node(model, G, current_node, target_node, visited, prediction_cache, debug=False):
    if debug:
        print(f"Predicting next node from current node: {current_node}")

    # Cache predictions to avoid redundant computations
    if current_node not in prediction_cache:
        target_features = G.nodes[target_node]["features"]
        predictions = []

        # Filter neighbors with valid features
        neighbors = [
            neighbor for neighbor in G.neighbors(current_node)
            if "features" in G.nodes[neighbor] and neighbor not in visited
        ]

        # Check if the target node is a direct neighbor
        if target_node in neighbors:
            if debug:
                print(f"Target node {target_node} is a direct neighbor. Moving to target.")
            return target_node

        # Predict probabilities for remaining neighbors
        for neighbor in neighbors:
            neighbor_features = G.nodes[neighbor]["features"]
            feature_vector = neighbor_features - target_features
            prob = model.predict(feature_vector.reshape(1, -1), verbose=0)[0][0]  # Suppress model output
            predictions.append((neighbor, prob))
        
        # Sort predictions by probability in descending order
        predictions.sort(key=lambda x: x[1], reverse=True)
        prediction_cache[current_node] = predictions

    # Select the next node with the highest probability
    next_node = None
    for neighbor, prob in prediction_cache[current_node]:
        if neighbor not in visited:
            next_node = neighbor
            break
    
    if debug:
        print(f"Next node chosen: {next_node}")
    return next_node




# Find the path with a limit on the number of hops
def find_path(model, G, source, target, max_hops=40, debug=False):
    if debug:
        print(f"Starting pathfinding from source: {source} to target: {target}, with max hops: {max_hops}")
    current_node = source
    visited = set()
    prediction_cache = {}  # Cache predictions to avoid recomputation
    path = [source]
    hops = 0

    while current_node != target:
        print(f"Current Number of Hops: {hops}")
        print(f"Current Node: {current_node}")
        visited.add(current_node)
        next_node = predict_next_node(model, G, current_node, target, visited, prediction_cache, debug=debug)
        if next_node is None:
            if debug:
                print(f"Pathfinding failed: no valid neighbors from {current_node}.")
            return None  # No path found
        path.append(next_node)
        current_node = next_node
        hops += 1

        if hops > max_hops:
            if debug:
                print(f"Pathfinding terminated: exceeded max hops ({max_hops}).")
            return None

    if debug:
        print(f"Pathfinding complete. Path: {path}")
    return path

In [20]:
def evaluate_pathfinding(model, G, max_hops=20, num_runs=20):
    total_hops = 0
    successful_runs = 0

    # Filter nodes with features
    nodes_with_features = [node for node in G.nodes if "features" in G.nodes[node]]

    for run in range(num_runs):
        # Ensure start and target nodes have features
        if len(nodes_with_features) < 2:
            print("Not enough nodes with features for evaluation.")
            break

        source_node, target_node = random.sample(nodes_with_features, 2)
        print(f"Run {run + 1}/{num_runs}: Source {source_node} -> Target {target_node}")
        
        # Call the pathfinding function
        path = find_path(model, G, source_node, target_node, max_hops)
        
        if path:
            num_hops = len(path) - 1  # Number of hops is the length of the path minus 1
            print(f"  Path found in {num_hops} hops. Path: {path}")
            total_hops += num_hops
            successful_runs += 1
        else:
            print("  No path found or run terminated.")
    
    if successful_runs > 0:
        average_hops = total_hops / successful_runs
        success_rate = successful_runs / num_runs * 100
    else:
        average_hops = float('inf')  # No successful runs
        success_rate = 0.0
    
    print(f"\n--- Summary ---")
    print(f"Success rate: {success_rate:.2f}% ({successful_runs}/{num_runs})")
    print(f"Average hops: {average_hops:.2f}" if successful_runs > 0 else "No successful runs.")
    
    return success_rate, average_hops


# Run evaluation
success_rate, average_hops = evaluate_pathfinding(model, G, max_hops=20, num_runs=20)

Run 1/20: Source 22723 -> Target 6289
Current Number of Hops: 0
Current Node: 22723
Current Number of Hops: 1
Current Node: 3307
Current Number of Hops: 2
Current Node: 4597
Current Number of Hops: 3
Current Node: 3849
Current Number of Hops: 4
Current Node: 3339
Current Number of Hops: 5
Current Node: 9072
Current Number of Hops: 6
Current Node: 6891
Current Number of Hops: 7
Current Node: 8995
Current Number of Hops: 8
Current Node: 7078
Current Number of Hops: 9
Current Node: 8300
Current Number of Hops: 10
Current Node: 3476
Current Number of Hops: 11
Current Node: 3178
Current Number of Hops: 12
Current Node: 3310
Current Number of Hops: 13
Current Node: 17087
Current Number of Hops: 14
Current Node: 16410
Current Number of Hops: 15
Current Node: 10189
Current Number of Hops: 16
Current Node: 13585
Current Number of Hops: 17
Current Node: 801
Current Number of Hops: 18
Current Node: 806
Current Number of Hops: 19
Current Node: 22827
Current Number of Hops: 20
Current Node: 799
  N

KeyboardInterrupt: 